we want something more term-centric than field-centric. This is what our previous query looks like under the covers:

In [1]:
curl -XGET 'localhost:9200/_validate/query?explain&pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "multi_match": {
      "query":   "Poland Street W1V",
      "type":    "most_fields",
      "fields":  [ "street", "city", "country", "postcode" ]
    }
  }
}
'

{
  "valid" : true,
  "_shards" : {
    "total" : 10,
    "successful" : 10,
    "failed" : 0
  },
  "explanations" : [
    {
      "index" : ".kibana",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (country:poland country:street country:w1v) (postcode:poland postcode:street postcode:w1v) (street:poland street:street street:w1v)"
    },
    {
      "index" : "my_index",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (country:poland country:street country:w1v) (postcode:poland postcode:street postcode:w1v) (street:poland street:street street:w1v)"
    },
    {
      "index" : "my_temp_index",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (country:poland country:street country:w1v) (postcode:poland postcode:street postcode:w1v) (street:poland street:street street:w1v)"
    },
    {
      "index" : "gb",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (count

matching poland in two fields could match higher than poland street matching in one field. We don't want that

try using the "and" operator to trim the long tail of results however

In [2]:
curl -XGET 'localhost:9200/_validate/query?explain&pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "multi_match": {
      "query":   "Poland Street W1V",
      "type":    "most_fields",
      "fields":  [ "street", "city", "country", "postcode" ]
    }
  }
}
'

{
  "valid" : true,
  "_shards" : {
    "total" : 10,
    "successful" : 10,
    "failed" : 0
  },
  "explanations" : [
    {
      "index" : ".kibana",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (country:poland country:street country:w1v) (postcode:poland postcode:street postcode:w1v) (street:poland street:street street:w1v)"
    },
    {
      "index" : "my_index",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (country:poland country:street country:w1v) (postcode:poland postcode:street postcode:w1v) (street:poland street:street street:w1v)"
    },
    {
      "index" : "my_temp_index",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (country:poland country:street country:w1v) (postcode:poland postcode:street postcode:w1v) (street:poland street:street street:w1v)"
    },
    {
      "index" : "blogs",
      "valid" : true,
      "explanation" : "(city:poland city:street city:w1v) (co

all three of the words must match a given field to return any results. Still not what we want

here is a use case where a more rare term showing in one field will score higher than an exact match

In [4]:
curl -XGET 'localhost:9200/my_index/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "query": {
        "multi_match": {
            "query":       "Peter Smith",
            "type":        "most_fields",
            "fields":      [ "*_name" ]
        }
    }
}
'

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 0,
    "max_score" : null,
    "hits" : [ ]
  }
}


Smith Williams will score higher than Peter Smith!

The solution is to add a third composite field for first and last name: full_name and search on that

however, we don't care much for redundant data. Onto the next section